# Drawing Random Numbers

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Uniform

In [ ]:
%matplotlib notebook

low, high = 0, 1
shape = (10000,)
samples = np.random.uniform(low, high, shape)
plt.hist(samples, bins=50);

## Normal

In [ ]:
%matplotlib notebook

mu, sigma = 0, 1
shape = (10000,)
samples = np.random.normal(mu, sigma, shape)
plt.hist(samples, bins=50);

# Sampling From Cumulative Distribution Function (C.D.F.)

## The exponential distribution

$$pdf(x) = \lambda e^{-\lambda x} $$

In [ ]:
%matplotlib notebook

lam = 1
x = np.arange(0, 10, 0.1)

pdf = lam * np.exp(-lam * x)
plt.plot(x, pdf)
plt.xlabel('x')
plt.ylabel('pdf(x)');

### Find the C.D.F.

$$ \int_{0}^{x} pdf(y) dy = 1 - e^{-\lambda x} $$

In [ ]:
%matplotlib notebook

cdf = 1-np.exp(-lam*x)
plt.plot(x, cdf)
plt.xlabel('x')
plt.ylabel('cdf(x)');

Since the c.d.f. always ranges from 0 to 1, if we chose some uniform random number within this interval, the chance of it occuring within some subrange from $a$ to $b$ is $a-b$.

In [ ]:
%matplotlib notebook

a = 0.2
b = 0.1

cdf = 1-np.exp(-lam*x)
plt.plot(x, cdf)

for i in range(100):
    sample = np.random.random()
    if a > sample > b:
        color = 'r'
    else:
        color = 'k'
    plt.axhline(sample, color=color, alpha=0.5, lw=1)

plt.xlabel('x')
plt.ylabel('cdf(x)');

So these samples of the c.d.f. value can be inverted to sample the values that occur within some probability interval.  We do this by inverting the c.d.f. to find the *quantile* function which maps a uniform random number to a number sampled from our distribution.

$$ Q(p) = -\frac{ln(1-p)}{\lambda} $$

In [ ]:
%matplotlib notebook

p = np.random.uniform(low, high, shape)
samples = -np.log(1-p)/lam
plt.plot(x, pdf, label='cdf(x)')
plt.hist(samples, bins=50, density=True)
plt.xlabel('x');

# Monte Carlo Methods

## Computing Pi

In [ ]:
%matplotlib notebook

R = 1
L = 2
fig, ax = plt.subplots(1,1, figsize=(8,8))
ax.add_patch(plt.Rectangle((-1,-1), L, L, fc='none', ec='k'))
ax.add_patch(plt.Circle((0, 0), radius=R, fc='none', ec='r'))
ax.set_xlim(-1.5,1.5)
ax.set_ylim(-1.5, 1.5);

## Area of Rectangle

$$A_{rect} = L^{2}$$

## Area of the Circle

$$A_{circ} = \pi R^{2}$$

## Relating the two

$$L = 2R$$

$$\frac{A_{circ}}{A_{rect}} = \frac{\pi R^{2}}{L^{2}} = \frac{\pi R^{2}}{4 R^{2}} = \frac{\pi}{4} $$

So if we can estimate the ratio of areas of the two shapes, we can estimate $\pi$!  We can get an idea of the area the circle takes up by randomly dropping points around it.  The points that lie inside the circle will be within the area of the circle.  If we randomly sample over the bounds of the rectangle, the total number of samples will be proportional to the area of the rectangle.

To determine if a random point, $(x, y)$ lies within a circle, we simply check if the magnitude of the vector is less than the radius of the circle.

$$x^{2} + y^{2} < R^{2}$$

In [ ]:
%matplotlib notebook

N_samples = 5000
shape = (2, N_samples)
x, y = samples = np.random.uniform(-1, 1, shape)
mask = x**2 + y**2 < R**2

fig, ax = plt.subplots(1,1, figsize=(8,8))
ax.add_patch(plt.Rectangle((-1,-1), L, L, fc='none', ec='k'))
ax.add_patch(plt.Circle((0, 0), radius=R, fc='none', ec='r'))
ax.set_xlim(-1.5,1.5)
ax.set_ylim(-1.5, 1.5)


plt.scatter(x, y, edgecolor='none', marker='o', s=5)
plt.scatter(x[mask], y[mask], edgecolor='none', marker='o', s=5)

pi_est = samples[:, mask].shape[1]/samples.shape[1]*4
plt.title(r'$\pi \approx {:.4f}$'.format(pi_est));

## Improving Accuracy

In [ ]:
%matplotlib notebook
import numba

@numba.vectorize
def estimate_pi(N_samples):

    shape = (2, int(N_samples))
    x, y = np.random.uniform(-1, 1, shape)
    mask = x**2 + y**2 < R**2
    pi_est = mask.sum()/N_samples*4
    return pi_est
    
N_samples = np.logspace(1, 16, 16, base=2, dtype=np.int64)
pi_est = estimate_pi(N_samples)


plt.plot(N_samples, pi_est, label='Estimate', color='orange')
plt.axhline(np.pi, label=r'Numpy $\pi$', ls='--')

plt.xscale('log')
plt.xlabel('N Samples')
plt.legend();

## We should do this statistically!

In [ ]:
%matplotlib notebook


@numba.vectorize
def estimate_pi(N_samples):

    shape = (2, int(N_samples))
    x, y = np.random.uniform(-1, 1, shape)
    mask = x**2 + y**2 < R**2
    pi_est = mask.sum()/N_samples*4
    return pi_est

@numba.njit
def estimate_pi_distribution(N_samples, N_iter):
    N_iter = int(N_iter)
    estimates = np.empty((N_samples.shape[0], N_iter))
    for i in range(N_samples.shape[0]):
        for j in range(N_iter):
            estimates[i, j] = estimate_pi(N_samples[i])
    return estimates

pi_estimate = estimate_pi_distribution(N_samples, 60)
pi_estimate_mean = pi_estimate.mean(axis=1)
pi_estimate_std = pi_estimate.std(axis=1)


plt.plot(N_samples, pi_estimate_mean, label='Mean', color='orange')
plt.fill_between(N_samples, pi_estimate_mean-pi_estimate_std, 
                 pi_estimate_mean+pi_estimate_std, color='orange', alpha=0.5)
plt.axhline(np.pi, label=r'Numpy $\pi$', ls='--')

plt.xscale('log')
plt.xlabel('N Samples')
plt.legend();

## Let's watch how our error converges

In [ ]:
%matplotlib notebook
from scipy.stats import linregress

slope, intercept, *_ = linregress(np.log(N_samples), np.log(pi_estimate_std))

plt.plot(N_samples, pi_estimate_std, label='Mean', color='orange')

plt.plot(N_samples, np.exp(slope*np.log(N_samples)+intercept), label='Best Fit', ls='--')

plt.xscale('log')
plt.yscale('log')
plt.xlabel('N Samples')
plt.title(r'Convergence Goes as $N^{%.2f}$' % (slope))
plt.legend();